Cell 1 — Install required libraries

In [1]:
# Install required libraries
!pip install albumentations opencv-python boto3 tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 204.8 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 292.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.3/582.3 kB 62.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 109.2 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [numpy]
      Successfully uninstalled numpy-1.26.4━━━━━━━━━━━━━━━━━━━ 2/6 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [albumentations]m [albumentations]eadless]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.16.2 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.2.6 which is incompatible.


Cell 2 — Import libraries

In [9]:
# OS utilities
import os

# Random selection
import random

# OpenCV for image processing
import cv2

# Numerical operations
import numpy as np

# Albumentations for data augmentation
import albumentations as A

# AWS SDK for Python
import boto3

# Progress bar
from tqdm import tqdm


STEP 3 — Initialize S3 client

In [10]:
# Create S3 client (AWS credentials must already be configured)
s3 = boto3.client("s3")


Cell 4 — Define S3 path

In [11]:
# S3 bucket name
S3_BUCKET = "ai-bmi-predictor-v2"

# Source images path (original dataset)
S3_SOURCE_BASE = (
    "tight and loose classifier/"
    "orginal dataset/"
    "Dataset/"
    "Training Data"
)

# Target path for synthetic images
S3_TARGET_BASE = (
    "tight and loose classifier/"
    "synthetic data/"
    "Training Data"
)

# Number of synthetic images per class
TARGET_IMAGES_PER_CLASS = 1000


Cell 5 — Define augmentation pipeline

In [12]:
# Classical augmentation pipeline
augmentation_pipeline = A.Compose([

    # Slight brightness and contrast changes
    A.RandomBrightnessContrast(
        brightness_limit=0.1,
        contrast_limit=0.1,
        p=0.8
    ),

    # Very small color changes
    A.HueSaturationValue(
        hue_shift_limit=5,
        sat_shift_limit=10,
        val_shift_limit=5,
        p=0.6
    ),

    # Tiny random noise
    A.GaussNoise(
        var_limit=(5.0, 15.0),
        p=0.5
    ),

    # Slight blur
    A.GaussianBlur(
        blur_limit=(3, 5),
        p=0.4
    )
])


/tmp/ipykernel_6041/1217378263.py:20: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(


Cell 6 — List images from an S3 folder

In [6]:
# Function to upload image to S3
def upload_to_s3(image, bucket, s3_key):
    
    # Encode image as JPEG
    success, buffer = cv2.imencode(".jpg", image)

    # Upload encoded image to S3
    s3.put_object(
        Bucket=bucket,
        Key=s3_key,
        Body=buffer.tobytes(),
        ContentType="image/jpeg"
    )


Cell 7 — Generate & upload synthetic images

In [13]:
# List image files from S3 prefix
def list_s3_images(bucket, prefix):

    image_keys = []

    paginator = s3.get_paginator("list_objects_v2")

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            if obj["Key"].lower().endswith((".jpg", ".jpeg", ".png")):
                image_keys.append(obj["Key"])

    print(f"📂 Found {len(image_keys)} images in {prefix}")
    return image_keys


STEP 7 — Load image from S3

In [14]:
# Load image from S3 into OpenCV
def load_image_from_s3(bucket, key):

    # Download object
    response = s3.get_object(Bucket=bucket, Key=key)

    # Read bytes
    image_bytes = response["Body"].read()

    # Convert bytes to numpy array
    image_array = np.frombuffer(image_bytes, np.uint8)

    # Decode image
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    return image


STEP 8 — Upload image to S3

In [15]:
# Upload OpenCV image to S3
def upload_to_s3(image, bucket, key):

    # Encode image as JPEG
    success, buffer = cv2.imencode(".jpg", image)

    # Upload to S3
    s3.put_object(
        Bucket=bucket,
        Key=key,
        Body=buffer.tobytes(),
        ContentType="image/jpeg"
    )


STEP 9 — Generate & upload augmented images

In [16]:
# Generate and upload synthetic images for a class
def generate_and_upload(class_name):

    print(f"\n🚀 Starting augmentation for class: {class_name}")

    # Build source prefix
    source_prefix = f"{S3_SOURCE_BASE}/{class_name}/"

    # Get list of original images
    source_keys = list_s3_images(S3_BUCKET, source_prefix)

    if len(source_keys) == 0:
        raise RuntimeError(f"No images found for class {class_name}")

    generated_count = 0

    # Progress bar
    with tqdm(total=TARGET_IMAGES_PER_CLASS, desc=f"Generating {class_name}") as pbar:

        while generated_count < TARGET_IMAGES_PER_CLASS:

            # Pick random source image
            source_key = random.choice(source_keys)

            # Load image
            image = load_image_from_s3(S3_BUCKET, source_key)

            if image is None:
                print(f"⚠️ Failed to load {source_key}")
                continue

            # Apply augmentation
            augmented = augmentation_pipeline(image=image)
            augmented_image = augmented["image"]

            # Build target S3 key
            target_key = (
                f"{S3_TARGET_BASE}/{class_name}/"
                f"{class_name}_{generated_count}.jpg"
            )

            # Print progress every 50 images
            if generated_count % 50 == 0:
                print(f"⬆️ Uploading {generated_count}/{TARGET_IMAGES_PER_CLASS}")

            # Upload augmented image
            upload_to_s3(augmented_image, S3_BUCKET, target_key)

            generated_count += 1
            pbar.update(1)

    print(f"✅ Completed augmentation for class: {class_name}")


In [17]:
# Generate synthetic data for Tight
generate_and_upload("Tight")

# Generate synthetic data for Loose
generate_and_upload("Loose")



🚀 Starting augmentation for class: Tight
📂 Found 46 images in tight and loose classifier/orginal dataset/Dataset/Training Data/Tight/


Generating Tight:   0%|          | 1/1000 [00:00<01:54,  8.75it/s]

⬆️ Uploading 0/1000


Generating Tight:   5%|▌         | 52/1000 [00:05<01:40,  9.44it/s]

⬆️ Uploading 50/1000


...

⬆️ Uploading 900/1000


Generating Loose:  95%|█████████▌| 953/1000 [01:10<00:03, 13.18it/s]

⬆️ Uploading 950/1000


Generating Loose: 100%|██████████| 1000/1000 [01:13<00:00, 13.58it/s]

✅ Completed augmentation for class: Loose
